In [1]:
import dask.bag as db

Helper function that displays all the functions that can be invoked on an object.

In [2]:
def display_object_method(obj):
    object_methods = [method_name for method_name in dir(obj)
                  if callable(getattr(obj, method_name))]
    print(object_methods)

Takes an array of lists and filename as input and creates a json file from the data.

In [3]:
import json
def create_json_file(data, filename):
    json_object = dict(data)
    json_string = json.dumps(json_object)
    open("output/%s" % filename, "w").write(json_string)
    

Computes the top K words from the bag of words for each of the subproject.

In [4]:
def compute_top_k(words, k=40):
    word_frequencies = words.frequencies()
    top40_words = word_frequencies.topk(k, key=1)
    word_counts = top40_words.compute()
    print(word_counts)
    return word_counts

## Subproject 1
### Top 40 words across all files
- Find top 40 words
- No need to filter out stop words

In [5]:
booklist = [
    "handout/data/pg36.txt",
    "handout/data/pg3207.txt",
    "handout/data/4300-0.txt",
    "handout/data/pg19033.txt",
    "handout/data/pg1497.txt",
    "handout/data/pg42671.txt",
    "handout/data/pg514.txt",
    "handout/data/pg6130.txt"
]
word_bag = db.read_text(booklist)

In [6]:
def get_lowercase_words(word_bag):
    stripped_words = word_bag.str.strip()
    split_words = stripped_words.str.split(" ")
    word_array = split_words.flatten().filter(lambda x: x!= "")
    return word_array.map(lambda x: x.lower())

In [7]:
lowercase_words = get_lowercase_words(word_bag)

In [8]:
top_40 = compute_top_k(lowercase_words, k=40)

[('the', 78837), ('and', 45168), ('of', 44739), ('to', 33436), ('a', 24234), ('in', 22126), ('that', 14818), ('he', 13019), ('is', 12918), ('his', 12270), ('i', 11044), ('with', 10296), ('for', 10036), ('as', 9639), ('be', 8834), ('was', 8787), ('not', 8141), ('it', 8123), ('but', 7856), ('by', 7701), ('or', 7407), ('her', 7403), ('they', 6735), ('which', 6517), ('you', 6354), ('on', 6214), ('from', 5811), ('at', 5695), ('are', 5590), ('she', 5458), ('all', 5437), ('their', 5285), ('have', 5146), ('had', 4647), ('this', 4090), ('my', 3841), ('so', 3710), ('we', 3629), ('no', 3620), ('if', 3571)]


In [9]:
create_json_file(top_40, "sp1.json")

## Subproject 2
### Top 40 filtered words
- Find top 40 words
- Filter out the stopwords from the handout file.

In [10]:
def get_filtered_words(words):
    stopwords = open("handout/data/stopwords.txt", "r").read().split("\n")
    return words.filter(lambda x: x not in stopwords)

In [11]:
filtered_words = get_filtered_words(lowercase_words)
top_40_filtered = compute_top_k(filtered_words, 40)

[('i', 11044), ('not', 8141), ('you', 6354), ('have', 5146), ('no', 3620), ('one', 3498), ('like', 2253), ('more', 2087), ('out', 2021), ('up', 1831), ('man', 1783), ('now', 1579), ('only', 1555), ('must', 1523), ('little', 1485), ('those', 1447), ('good', 1444), ('should', 1417), ('after', 1379), ('great', 1358), ('every', 1356), ('first', 1318), ('own', 1289), ('did', 1271), ('how', 1266), ('see', 1251), ('these', 1244), ('men', 1233), ('over', 1209), ('where', 1205), ('make', 1196), ('upon', 1188), ('nor', 1181), ('never', 1177), ('much', 1167), ('time', 1166), ('said,', 1163), ('two', 1142), ('old', 1140), ('made', 1128)]


In [12]:
create_json_file(top_40_filtered, "sp2.json")

## Subproject 3
### Top 40 words without punctuations
- Find top 40 words
- Remove leading and trailing punctuation marks
- Filter out the stopwords from the handout file.

In [21]:
from string import punctuation

def get_words_without_punctuations(lowercase_words):
    all_punctuations = ".,:;'!?"
    return lowercase_words.map(lambda x: x.strip(all_punctuations))

In [29]:
long_words = filtered_words.filter(lambda x: len(x) > 1)
words_without_punctuations = get_words_without_punctuations(long_words)

# filtered_words = get_filtered_words(long_words)
top_40_without_punctuations = compute_top_k(words_without_punctuations, 40)

[('not', 8664), ('you', 7262), ('have', 5258), ('one', 3959), ('no', 3888), ('man', 2610), ('more', 2404), ('like', 2378), ('out', 2322), ('up', 2194), ('now', 2106), ('men', 1860), ('good', 1839), ('mr', 1789), ('only', 1706), ('time', 1701), ('god', 1666), ('first', 1646), ('say', 1616), ('must', 1571), ('little', 1557), ('own', 1534), ('those', 1514), ('see', 1499), ('great', 1438), ('after', 1437), ('should', 1429), ('did', 1401), ('us', 1379), ('these', 1366), ('every', 1361), ('before', 1344), ('over', 1342), ('know', 1335), ('how', 1315), ('much', 1310), ('same', 1306), ('where', 1277), ('two', 1266), ('made', 1244)]


In [30]:
create_json_file(top_40_without_punctuations, "sp3.json")

## Subproject 4
### TF-IDF Calculation

In [24]:
def get_words_for_document(document):
    word_bag = db.read_text(document)
    lowercase_words = get_lowercase_words(word_bag)
    words_without_punctuations = get_words_without_punctuations(lowercase_words)
    return get_filtered_words(words_without_punctuations)    

In [25]:
import math

def get_IDF(words_by_document):
    unique_words = []
    for words_for_single_document in words_by_document:
        unique_words.append(words_for_single_document.distinct())
    
    large_bag = db.zip(*unique_words)
    flattened_bag = large_bag.flatten()
    frequencies = flattened_bag.frequencies()
    idf = frequencies.map(lambda x: (x[0], math.log(8/x[1])))
    return idf

In [26]:
def get_top5_TF_IDF(tf, idf):
    tf = tf.frequencies()
    tf_idf = tf.join(idf, lambda x: x[0], lambda x: x[0]).map(lambda x: (x[0][0], x[0][1] * x[1][1]))
    return tf_idf.topk(5, key=1).compute()

In [27]:
book_terms = []
for book in booklist:
    book_terms.append(get_words_for_document(book))

IDF = get_IDF(book_terms)

tf_idf = []
for terms in book_terms:
    tf_idf.extend(get_top5_TF_IDF(terms, IDF))

print(tf_idf)

[('martians', 330.6312051270939), ('pit', 164.27588179270703), ('martian', 160.11699870934734), ('cylinder', 131.00481712582965), ('hill', 118.52816787575064), ('soveraign', 1023.0852385064792), ('hath', 927.4309275892067), ('law', 902.4776290890487), ('common-wealth', 867.1271228804915), ('onely', 856.7299151720923), ('stephen', 1000.211381548001), ('john', 401.3322175442083), ('don’t', 363.90226979397124), ('dedalus', 332.71064666877373), ('mr', 331.3525586182436), ('alice', 345.1872959188527), ('gutenberg-tm', 116.4487263340708), ('rabbit', 62.383246250395075), ('electronic', 56.144921625355565), ('[illustration]', 45.747713916956386), ('justice', 496.2933812809208), ('yes', 456.0856026504527), ('plato', 447.7730786417247), ('evil', 313.30252561309527), ('unjust', 295.2806989185367), ('elizabeth', 804.0507294495366), ('darcy', 711.1690072545039), ('bennet', 578.0847485869944), ('jane', 526.0987100449985), ('bingley', 494.9070869198009), ('jo', 2476.6148761406844), ('meg', 1278.85654

In [28]:
create_json_file(tf_idf, "sp4.json")